BLDG 5301 Project

In [ ]:
# Load all the provided CSV files
weather_data_path = "weather.csv"
ahu1_data_path = "canal ahu 1.csv"
ahu2_data_path = "canal ahu 2.csv"
energy_data_path = "canal energy use.csv"

# Read the files into pandas DataFrames
weather_data = pd.read_csv(weather_data_path)
ahu1_data = pd.read_csv(ahu1_data_path)
ahu2_data = pd.read_csv(ahu2_data_path)
energy_data = pd.read_csv(energy_data_path)

# Rename and convert timestamp columns to datetime format for all datasets
weather_data.rename(columns={"Unnamed: 0": "Timestamp"}, inplace=True)
weather_data["Timestamp"] = pd.to_datetime(weather_data["Timestamp"])

ahu1_data.rename(columns={"Unnamed: 0": "Timestamp"}, inplace=True)
ahu1_data["Timestamp"] = pd.to_datetime(ahu1_data["Timestamp"])

ahu2_data.rename(columns={"Unnamed: 0": "Timestamp"}, inplace=True)
ahu2_data["Timestamp"] = pd.to_datetime(ahu2_data["Timestamp"])

energy_data.rename(columns={"Unnamed: 0": "Timestamp"}, inplace=True)
energy_data["Timestamp"] = pd.to_datetime(energy_data["Timestamp"])

# Set the Timestamp column as the index for all datasets
weather_data.set_index("Timestamp", inplace=True)
ahu1_data.set_index("Timestamp", inplace=True)
ahu2_data.set_index("Timestamp", inplace=True)
energy_data.set_index("Timestamp", inplace=True)

# Identify missing timestamps for each dataset compared to the weather data
missing_ahu1 = weather_data.index.difference(ahu1_data.index)
missing_ahu2 = weather_data.index.difference(ahu2_data.index)
missing_energy = weather_data.index.difference(energy_data.index)

# Print the missing timestamps neatly
print("Missing Timestamps in AHU 1 data:")
print(missing_ahu1)

print("\nMissing Timestamps in AHU 2 data:")
print(missing_ahu2)

print("\nMissing Timestamps in Energy Use data:")
print(missing_energy)


Missing Timestamps in AHU 1 data:
DatetimeIndex(['2017-12-31 01:00:00', '2017-12-31 02:00:00',
               '2017-12-31 03:00:00', '2017-12-31 04:00:00',
               '2017-12-31 05:00:00', '2017-12-31 06:00:00',
               '2017-12-31 07:00:00', '2017-12-31 08:00:00',
               '2017-12-31 09:00:00', '2017-12-31 10:00:00',
               '2017-12-31 11:00:00', '2017-12-31 12:00:00',
               '2017-12-31 13:00:00', '2017-12-31 14:00:00',
               '2017-12-31 15:00:00', '2017-12-31 16:00:00',
               '2017-12-31 17:00:00', '2017-12-31 18:00:00',
               '2017-12-31 19:00:00', '2017-12-31 20:00:00',
               '2017-12-31 21:00:00', '2017-12-31 22:00:00',
               '2017-12-31 23:00:00'],
              dtype='datetime64[ns]', name='Timestamp', freq=None)

Missing Timestamps in AHU 2 data:
DatetimeIndex(['2017-12-31 01:00:00', '2017-12-31 02:00:00',
               '2017-12-31 03:00:00', '2017-12-31 04:00:00',
               '2017-12-31 05:00

In [ ]:
import pandas as pd

# Load the uploaded CSV file
file_path = "canal energy use.csv"  # Replace with the actual path to your input file
data = pd.read_csv(file_path)

# Rename the timestamp column for clarity
# Assuming the timestamp column is unnamed, we rename it to 'Timestamp'
data.rename(columns={"Unnamed: 0": "Timestamp"}, inplace=True)

# Calculate the 'Total Chiller' column by summing 'Chiller 1' and 'Chiller 2'
data["Total Chiller"] = data["Chiller 1"] + data["Chiller 2"]

# Calculate the 'Total AHU' column by summing 'AHU 1' and 'AHU 2'
data["Total AHU"] = data["AHU 1"] + data["AHU 2"]

# Calculate the 'Total Plugs' column by summing all the plug columns
plug_columns = ["Plug 1", "Plug 2", "Plug 3", "Plug 4", "Plug 5", "Plug 6"]
data["Total Plugs"] = data[plug_columns].sum(axis=1)

# Calculate the 'Total Lighting' column by summing all the lighting columns
lighting_columns = [
    "Lighting 1",
    "Lighting 2",
    "Lighting 3",
    "Lighting 4",
    "Lighting 5",
    "Lighting 6",
]
data["Total Lighting"] = data[lighting_columns].sum(axis=1)

# Create a new DataFrame with only the required columns
cleaned_data = data[
    ["Timestamp", "Total Chiller", "Total AHU", "Total Plugs", "Total Lighting"]
]

# Save the new data to an Excel file
output_file_path = "Canal_Clean_Energy_Use.xlsx"  # Specify your desired output path
cleaned_data.to_excel(output_file_path, index=False)

print(f"New Excel file created: {output_file_path}")


New Excel file created: Canal_Clean_Energy_Use.xlsx
